<h3>Introduction</h3>

<h5>Dataset: 
<br>The input data comprises of files within the folders with class labels as their name</br>
<br>The textual data are emails written from one journalist to another or to their source, regarding a story.</br>
<br>Emails can be seen labelled under multiple classes which would mislead the model.</br></h5>

<h2>Q1: Collect the data set and import necessary libraries (0.5 Marks) </h2>

In [1]:
# import requried libraries
# .............. Dataset processsing libraries..................
import numpy as np
import pandas as pd
import os
# ............... Text processing libraries ....................
import regex
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer
from string import punctuation
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
import re
from collections import defaultdict
import json

<h1> Data preprocessing </h1>

In [2]:
#unzip the file (downloaded from the kaggle)
if not os.path.exists("Data"):
    from zipfile import ZipFile
    archive = "archive.zip"
    with ZipFile(archive, 'r') as zip:
       zip.extractall()

# load the file location
path = os.getcwd()
path = os.path.join(path, 'Data')
[x[0] for x in os.walk(path)]

['D:\\Bits\\IR Assignment 1\\Data',
 'D:\\Bits\\IR Assignment 1\\Data\\Crime',
 'D:\\Bits\\IR Assignment 1\\Data\\Entertainment',
 'D:\\Bits\\IR Assignment 1\\Data\\Politics',
 'D:\\Bits\\IR Assignment 1\\Data\\Science',
 'D:\\Bits\\IR Assignment 1\\Data\\Science\\.ipynb_checkpoints']

In [3]:
# print the sub-directory names
directory_contents = os.listdir(path)
print(directory_contents)

['Crime', 'Entertainment', 'Politics', 'Science']


In [4]:
# Get the number of files in each sub-directory (topics)
files = {}
unique = []
for name in directory_contents:
    # base = path + '\\' +name
    base = os.path.join(path,name)
    x = []
    with os.scandir(base) as entries:
        for entry in entries:
            if(entry.is_file()):
                x.append(os.path.join(base,entry.name))
    files[name] = x
    
length = 0
for k, v in files.items():
    print(k, len(v))
    length = length+len(v)

Crime 1100
Entertainment 1053
Politics 3001
Science 4000


<h2> Question 2: Demonstrate stopwords removal process (0.5 Marks) </h2>

In [5]:
# setting it up
nltk.download('stopwords')
stop = stopwords.words('english')
ps = SnowballStemmer(language = 'english')
for punct in punctuation:
    stop.append(punct)
    
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

def removing_special_characters(words):
    mod_words = []
    symbols = [ "'",'/','.','-','!','@','#','$','^','&','*','(',')','+','=','<','>','?','_','"']
    #words = [word.replace(sym,'') for word in words for sym in symbols if sym in word]
    removed_symbols = []
    for word in words :
        for sym in symbols:
            if sym in word:
                word = word.replace(sym,'')
        if word!="":
            removed_symbols.append(word)
    return removed_symbols

# remove stop words and lemitiz and tokenize
def filter_text(text, stop_words):
    word_tokens = WordPunctTokenizer().tokenize(text.lower())
    word_tokens = removing_special_characters(word_tokens)
    filtered_text = [wordnet_lemmatizer.lemmatize(w, pos="v") for w in word_tokens if not w in stop_words]
    return " ".join(filtered_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Abhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Abhi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


<h2> Question 3: Tokenize the terms in the considered documents and demonstrate normalization using an appropriate stemmer (1Mark) </h2>

<h4>Answer: We are using snowball stemmer because of following reasons</h4>
<br> 1. More practical accuracy </br>
<br> 2. Improved rules over porter  and faster computation time</br>
<br> 3. It has Reasonably large community to find the quick support and less error prone </br>

<h2> Create an inverted index that can used for Boolean queries(2 Marks) </h2>

In [6]:
# make inverted index
def inverted_index(stop_words,files):
    dictionary = {}
    documents = {}
    doc_no = 0
    for file1 in files:
        print(file1)
        for file in files[file1]:
            doc_no = doc_no + 1
            with open (file,'r') as file:
                #next(file)
                s=file.read().replace('\n',' ')

            s = filter_text(s,stop_words)
            key = 'file_' + str(doc_no)

            documents.setdefault(key,[])
            documents[key].append(s)

            stemmed = []

            #stemming
            for i in s.split():
                stemmed.append(ps.stem(i))

            #creating posting list
            for x in stemmed:
                key = x
                dictionary.setdefault(key, [])
                dictionary[key].append(doc_no)

            #removing duplicates
            dictionary = {a:list(set(b)) for a, b in dictionary.items()}

    return dictionary,documents

<h2> Create a positional index that can be used for Phrase queries and proximity queries.(2 Marks) </h2>

In [7]:
# make Positional index
def positional_index(stop_words,files):
 
    dictionary = {}
    documents = {}
    doc_no = 0
    for file1 in files:
        for file in files[file1]:
            doc_no = doc_no + 1
            with open (file,'r') as file:
                s=file.read().replace('\n',' ')[1:]
            s = filter_text(s,stop_words)
            key = 'file_' + str(doc_no)
            documents.setdefault(key,[])
            documents[key].append(s)

            temp_dict = {}
            stemmed = []

            #stemming
            for i in s.split():
                stemmed.append(ps.stem(i))

            #creating positional index posting lists
            a = 0
            for x in stemmed:
                key = x
                temp_dict.setdefault(key, [])
                temp_dict[key].append(a)
                a += 1

            for x in temp_dict:
                if dictionary.get(x):
                    dictionary[x][doc_no] = temp_dict.get(x)
                else:
                    key = x
                    dictionary.setdefault(key, [])
                    dictionary[key] = {}
                    dictionary[x][doc_no] = temp_dict.get(x)
 
    return dictionary,documents

In [8]:
# Inverted Index
dicti_inverted,doc_inverted = inverted_index(stop,files)

Crime
Entertainment
Politics
Science


In [9]:
dicti_inverted.keys()

dict_keys(['archiv', 'name', 'ripem', 'faq', 'last', 'updat', 'sun', '7', 'mar', '93', '21', '00', '0500', 'post', 'still', 'rather', 'rough', 'list', 'like', 'question', 'inform', 'program', 'public', 'key', 'mail', 'encrypt', 'write', 'maintain', 'marc', 'vanheyningen', 'mvanheyn', 'whale', 'cs', 'indiana', 'edu', 'varieti', 'newsgroup', 'month', 'basi', 'follow', 'discuss', 'specif', 'redirect', 'group', 'alt', 'secur', 'reformat', 'attempt', 'compli', 'standard', 'hypertext', 'format', 'allow', 'easi', 'manipul', 'document', 'world', 'wide', 'web', 'let', 'know', 'think', 'disclaim', 'noth', 'consid', 'legal', 'advic', 'anyth', 'one', 'person', 'opinion', 'want', 'real', 'talk', 'lawyer', 'answer', '1', 'perform', 'privaci', 'enhanc', 'pem', 'use', 'cryptograph', 'techniqu', 'rsa', 'des', 'electron', 'properti', 'authent', 'e', 'send', 'confirm', 'nobodi', 'read', 'except', 'intend', 'recipi', 'primarili', 'mark', 'riordan', 'mrr', 'scss3', 'cl', 'msu', 'code', 'domain', 'routin', 

In [10]:
# Positional Index
dicti_positional,doc_positional = positional_index(stop,files)

In [11]:
dicti_positional

{'archiv': {1: [0],
  2: [35, 88, 1042],
  3: [42],
  31: [29],
  34: [61, 418, 1687],
  35: [73,
   301,
   1111,
   1114,
   1794,
   1817,
   1827,
   1850,
   3797,
   4800,
   4802,
   4820,
   4852,
   4923,
   4930],
  36: [61, 4846],
  91: [182]},
 'name': {1: [1, 463],
  2: [36, 633],
  3: [43, 402, 442],
  4: [36],
  30: [7, 39, 82],
  34: [62,
   514,
   604,
   867,
   925,
   936,
   1031,
   1079,
   1083,
   1139,
   1407,
   1409,
   1428,
   1439,
   2181,
   2276,
   2408,
   2659,
   2660,
   2662,
   3474],
  35: [74, 440, 1112, 1507, 2559, 3377, 4950, 5560],
  36: [62, 774, 782, 1605, 1614, 1807, 2671, 2799, 2988, 3762, 3895, 4419],
  39: [267],
  44: [23, 33],
  56: [71],
  70: [70],
  82: [6],
  93: [98],
  97: [68]},
 'ripem': {1: [2,
   22,
   29,
   47,
   52,
   92,
   93,
   120,
   146,
   147,
   182,
   185,
   195,
   214,
   243,
   283,
   323,
   343,
   359,
   366,
   383,
   507,
   557,
   568,
   592,
   602,
   641,
   656,
   666,
   671,
   78

<h2> Show an implementation for  Boolean query processing and Proximity processing.(3 Marks) </h2>

In [12]:
def process_words_query(query,stop_words):
    #
    word_tokens = WordPunctTokenizer().tokenize(query.lower())
    word_tokens = removing_special_characters(word_tokens)
    filtered_text = [wordnet_lemmatizer.lemmatize(w, pos="v") for w in word_tokens if not w in stop_words]
    stemmed = []
    for i in filtered_text:
      stemmed.append(ps.stem(i))
    return stemmed

def and_merge(post_list1, post_list2):
    ans = []
    ptr1 = 0 ; ptr2 = 0
    comparisons = 0
    while ptr1 < len(post_list1) and ptr2 < len(post_list2):
        
        if post_list1[ptr1] == post_list2[ptr2]:
            ans.append(post_list1[ptr1])
            ptr1 += 1
            ptr2 += 1
        elif post_list1[ptr1] < post_list2[ptr2] :
            ptr1 += 1
        else :
            ptr2 += 1
        comparisons += 1
    return ans, comparisons

def or_merge(post_list1, post_list2):

    ans = []
    ptr1 = 0 ; ptr2 = 0
    comparisons = 0
    while ptr1 < len(post_list1) and ptr2 < len(post_list2):

        if post_list1[ptr1] == post_list2[ptr2]:
            ans.append(post_list1[ptr1])
            ptr1 += 1
            ptr2 += 1
        elif post_list1[ptr1] < post_list2[ptr2] :
            ans.append(post_list1[ptr1])
            ptr1 += 1
        else :
            ans.append(post_list2[ptr2])
            ptr2 += 1
        comparisons += 1
        
    while(ptr1 < len(post_list1)):
        ans.append(post_list1[ptr1])
        ptr1 += 1

    while(ptr2 < len(post_list2)):
        ans.append(post_list2[ptr2])
        ptr2 += 1
        
    return ans, comparisons


def not_merge(post_list1,doc_ID):
    result = []
    for doc in range(1,doc_ID):
        if doc not in post_list1 :
            result.append(doc)
    return result

def retrieve_list(term,temp_results):
    ans = []
    if term in dicti_inverted.keys():
        ans = dicti_inverted[term]
    elif term in temp_results.keys():
        ans = temp_results[term]
    else :
        print('Postings list is not there for given term ')
    return ans


def processing_not_operator(query):
    temp_results = {}
    ind = 0
    i = len(query) - 1
    while i >= 0:
        if query[i] == 'not' :
            if query[i-1] == 'not':
                del query[i]
                del query[i-1]
                i -= 1
            else:
                term = query[i+1]
                post_list = retrieve_list(term,temp_results)
                key = 'result'+str(ind)
                ind += 1
                result = not_merge(post_list,doc_ID)
                temp_results[key] = result

                query[i] = key
                del query[i+1]
        i -= 1
    return query, temp_results, ind

def process_MIX(query, temp_results, ind):
    
    comparisons = 0
    while True:
        
        if 'and' in query:
            index = query.index('and')
            term1 = query[index-1]
            term2 = query[index+1]

            list1 = retrieve_list(term1, temp_results)
            list2 = retrieve_list(term2, temp_results)

            res,comp = and_merge(list1, list2)
            comparisons += comp
            key = 'result'+str(ind)
            ind += 1
            temp_results[key] = res
            query[index] = key
            del query[index-1]
            del query[index] # because of re arrangement
            print(query)
            
        else :
            
            query = [term for term in query if term != 'or']
            while len(query) > 1:
                term1 = query[0]
                term2 = query[1]
                list1 = retrieve_list(term1, temp_results)
                list2 = retrieve_list(term2, temp_results)
                
                res,comp = or_merge(list1, list2)
                comparisons += comp
                
                key = 'result'+str(ind)
                ind += 1
                temp_results[key] = res
                query[0] = key
                del query[1]
            break
    results = retrieve_list(query[0], temp_results)
    return results,comparisons

def all_and_operators(query):
    ans = True
    if 'or' in query :
        ans = False
    return ans
            
def find_minimum(lists):
    
    dummy = lists.copy()
    pos1 = lists.index(min(lists))
    del dummy[pos1]
    sec_min = min(dummy)
    pos2 = lists.index(sec_min)
    
    return pos1,pos2

def optimised_AND(query, temp_results, ind):
    results = []
    comparisons = 0
    if len(query) == 0 :
        print('Nothing to Process')
    else :
        print('OPTIMISING AND OPERATION')
        query = [term for term in query if term != 'and']
        posts_length = []
        for term in query:
            posts_length.append(len(retrieve_list(term,temp_results)))
        
        while True :
            
            if len(query) == 1:
                results = retrieve_list(query[0], temp_results)
                break
        
            else :
                print(posts_length)
                pos1,pos2 = find_minimum(posts_length)
                list1 = retrieve_list(query[pos1], temp_results)
                list2 = retrieve_list(query[pos2], temp_results)
                print('Performing AND between lists with length {} and {}'.format(len(list1),len(list2)))
                
                res,comp = and_merge(list1, list2)
                comparisons += comp
                key = 'result'+str(ind)
                ind += 1
                temp_results[key] = res
                posts_length[pos1] = len(res)
                query[pos1] = key
                
                del posts_length[pos2]
                del query[pos2]
    
    return results,comparisons
    
def process_query(query):
    
    temp_results = {}
    results = []
    query = process_words_query(query,stop)
    print('***final query after preprocessing***')
    print(query)
    print('***Processing NOT Operator***')
    query, temp_results, ind = processing_not_operator(query)
    print(query)
    
    if all_and_operators(query) :
        results,comparisons = optimised_AND(query,temp_results, ind)
        
    else :
        print('**** MIX of AND - OR****')
        results,comparisons = process_MIX(query, temp_results, ind)
        
    print('#####Total Comparisons : {}#####'.format(comparisons))
    return results

def print_results(results):
    docs = len(results)
    print('Total Docs Retrieved are : {}'.format(docs))
    files = []
    for id in results:
        files.append(doc_inverted["file_{}".format(id)])
    return files   
    
def read_query():
    query = input()
    results = process_query(query)
    files = print_results(results)
    return files



In [13]:
files = read_query()

 archive and product


***final query after preprocessing***
['archiv', 'product']
***Processing NOT Operator***
['archiv', 'product']
OPTIMISING AND OPERATION
[8, 20]
Performing AND between lists with length 8 and 20
#####Total Comparisons : 23#####
Total Docs Retrieved are : 4


In [15]:
files

[]

In [16]:
doc_inverted

{'file_1': ['archive name ripem faq last update sun 7 mar 93 21 00 00 0500 post still rather rough list likely question information ripem program public key mail encryption faq ripem write maintain marc vanheyningen mvanheyn whale cs indiana edu post variety newsgroups monthly basis follow discussion specific ripem redirect group alt security ripem month reformatted post attempt comply standards hypertext faq format allow easy manipulation document world wide web let know think disclaimer nothing faq consider legal advice anything one person opinion want real legal advice talk real lawyer question answer 1 ripem ripem program perform privacy enhance mail pem use cryptographic techniques rsa des allow electronic mail properties authentication e send confirm privacy e nobody read except intend recipient ripem write primarily mark riordan mrr scss3 cl msu edu code public domain except rsa routines library call rsaref license rsa data security inc 2 get ripem ripem contain library cryptogr

In [14]:
# Implementation of Proximity query process

def get_word_postings(word):
    preprocessed_word = str(preprocess(word, True))
    print(preprocessed_word)
    print("Frequency:",frequency[preprocessed_word][0])
    print("Postings List:",postings[preprocessed_word][0])
    
def get_positions(posting_values, doc):
    for posting_value in posting_values:
        if posting_value[0] == doc:
            return posting_value[1]
    return {}

def gen_init_set_matchings(word):
    init = []
    word_postings = postings[word]
    for word_posting in word_postings:
        for positions in word_postings[word_posting]:
            init.append((word_posting, positions))
    return init

def match_positional_index(init, b):
    matched_docs = []
    for p in init:
        doc = p[0]
        pos = p[1]

        count = 0

        for k in b:
            pos = pos+1
            k_pos = postings[k][0]
            docs_list = [z[0] for z in k_pos]
            if doc in docs_list:
                doc_positions = get_positions(k_pos, doc)
                if pos in doc_positions:
                    count += 1
                else:
                    count += 1
                    break

            if count == len(b):
                matched_docs.append(p[0])
    return set(matched_docs)

def run_query(query):
    processed_query = process_words_query(query,stop)
    print(processed_query)
    
    if len(query_tokens)==1:
        print("Total Document Mathces", [a[0] for a in postings[query][0]])
        return [a[0] for a in postings[query][0]]
    
    init_word = query_tokens[0]
    init_matches = gen_init_set_matchings(init_word)

    query_tokens.pop(0)
    total_matched_docs = match_positional_index(init_matches, query_tokens)
    print("Total Document Matches:", total_matched_docs)
    return total_matched_docs

def print_file(file):
    out_file = open(paths[file], 'r', encoding='cp1250')
    out_text = out_file.read()
    print(out_text)